In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import Config
import datetime
import time
import copy
import BS

from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score

from pymongo import MongoClient
client=MongoClient(Config.DB_Hostname,Config.DB_Port)

def _calc_spread_return(spot, long_start, long_end, short_start, short_end, strike_spread):
    
    spread_benefit = short_start - long_start
    margin = spot*0.02 - spread_benefit
    brokerage = (long_start + long_end + short_start + short_end) * 0.001
    
    if spread_benefit > 0:
        margin += strike_spread
        
    if margin <0:
        print(f"Error_{margin}")
    
    return (long_end - long_start + short_start - short_end - brokerage)/margin


class Gap_Move_Classifier:
    
    def __init__(self, start_date, end_date, prediction_date, underlying, start_holding_period, end_holding_period):
        
        self.start_date = start_date #2017-01-01
        self.end_date = end_date #2017-01-01
        self.prediction_date = prediction_date #2017-01-01
        self.underlying = underlying #NIFTY, BANKNIFTY
        self.underlying_dynamics = None
        self.vol_surface = None
        self.days_to_expiry = None
        self.data_matrix = None
        self.return_matrix = None
        self.price_matrix = None
        self.performance_stats = None
        self.start_holding_period = start_holding_period #in minutes
        self.end_holding_period = end_holding_period #in minutes
        self.strike_range = [-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1]
        
        #start = time.time()
        self.underlying_dynamics = pd.DataFrame(client[f'{Config.Data_DB}'][f'{self.underlying}OHLC'].find())
        self.underlying_dynamics.drop(columns=['_id'],inplace=True)
        self.underlying_dynamics.sort_values(['date','batch_id'],inplace=True)
        self.underlying_dynamics=self.underlying_dynamics[(self.underlying_dynamics.date>=self.start_date)&(self.underlying_dynamics.date<=self.end_date)]
        self.underlying_dynamics.batch_id=self.underlying_dynamics.batch_id.astype(int)
        self.underlying_dynamics.set_index('date',inplace=True)
        #print(f"Prices Downloaded in {time.time() - start} seconds")

        #start=time.time()
        self.vol_surface=pd.DataFrame(client[f'{Config.Data_DB}']['Vol_Surface'].find({"SYMBOL" : self.underlying}))
        self.vol_surface.drop(columns=['_id', 'CONTRACTS', 'VAL_INLAKH','OPEN_INT', 'CHG_IN_OI', 'SYMBOL'], inplace=True)
        self.vol_surface.sort_values('date',inplace=True)
        self.vol_surface=self.vol_surface[(self.vol_surface.date>=self.start_date)&(self.vol_surface.date<=self.end_date)]
        self.vol_surface.set_index('date',inplace=True)
        #print(f"Vol Surface Downloaded in {time.time() - start} seconds")
        
        #start=time.time()
        self.days_to_expiry = pd.DataFrame(client[f'{Config.Data_DB}']['Days_To_Expiry'].find({"underlying":self.underlying}))
        self.days_to_expiry.drop(columns = ['_id','underlying'],inplace=True)
        self.days_to_expiry.sort_values('date',inplace=True)
        self.days_to_expiry=self.days_to_expiry[(self.days_to_expiry.date>=self.start_date)&(self.days_to_expiry.date<=self.end_date)]
        self.days_to_expiry.set_index('date',inplace=True)
        #print(f"Expiry Dates Downloaded in {time.time()-start} seconds")

        #start=time.time()
        self._make_features()
        #print(f"Features Engineered in {time.time() - start} seconds")

        #start=time.time()
        self._generate_return_matrix()
        #print(f"Features Engineered in {time.time() - start} seconds")
                
    def _make_features(self):
        
        vol_surface = copy.deepcopy(self.vol_surface)
        if self.end_date not in vol_surface.index:
            vol_surface.loc[self.end_date] = [np.nan]*len(vol_surface.columns)

        closing_candle = self.underlying_dynamics[self.underlying_dynamics.batch_id==375 - (self.start_holding_period - 1)][['open']]
        if self.end_date not in closing_candle.index:
            closing_candle.loc[self.end_date] = np.nan
        closing_candle=closing_candle.shift()
        
        opening_candle = self.underlying_dynamics[self.underlying_dynamics.batch_id==1 + (self.end_holding_period - 1)][['close']]
        if self.end_date not in opening_candle.index:
            opening_candle.loc[self.end_date] = np.nan
        
        candle = pd.merge(closing_candle, opening_candle, left_index=True, right_index=True)
        candle.close = np.where(candle.close.isna(), candle.open, candle.close)
        
        self.data_matrix = pd.DataFrame()
        self.data_matrix["gap_move"] = (candle.close - candle.open) / candle.open
        self.data_matrix["gap_move_flag"] = np.where(self.data_matrix.gap_move>=0, 1, 0)
        self.data_matrix = pd.merge(self.data_matrix, vol_surface.shift(), left_index = True, right_index = True).dropna()
        
        candle = pd.merge(candle, self.days_to_expiry, left_index=True, right_index=True)
        candle = pd.merge(candle , 
                          self.days_to_expiry.shift().rename(columns = 
                            {"current_week" : "current_week_yesterday", "next_week": "next_week_yesterday"}), 
                          left_index=True,
                          right_index=True)
        
        candle.current_week_yesterday = np.where(candle.current_week_yesterday == 1, candle.next_week_yesterday, candle.current_week_yesterday)
        candle["IVOL"] = vol_surface["vol_0"].shift()
        candle.dropna(inplace = True)
        
        for strike_range in self.strike_range:
            
            candle[f"{strike_range}_strike_call_and_put"] = candle.open * (1 + strike_range/100) * np.exp(Config.interest_rate * (candle.current_week_yesterday - 1 + self.start_holding_period/1440) / 365)
            
            candle[f"call_{strike_range}_yesterday"] = candle.apply(lambda x : (BS.bs_call(x.open, 
                                                                                x[f"{strike_range}_strike_call_and_put"], 
                                                                                (x.current_week_yesterday - 1 + self.start_holding_period/1440)/ 365, 
                                                                                Config.interest_rate, 
                                                                                x.IVOL)), axis = 1)
            
            candle[f"call_{strike_range}_today"] = candle.apply(lambda x : (BS.bs_call(x.close, 
                                                                                x[f"{strike_range}_strike_call_and_put"], 
                                                                                (x.current_week - 0.75 - self.end_holding_period/1440)/ 365, 
                                                                                Config.interest_rate, 
                                                                                x.IVOL)), axis = 1)

            candle[f"put_{strike_range}_yesterday"] = candle.apply(lambda x : (BS.bs_put(x.open, 
                                                                                x[f"{strike_range}_strike_call_and_put"], 
                                                                                (x.current_week_yesterday - 1 + self.start_holding_period/1440)/ 365, 
                                                                                Config.interest_rate, 
                                                                                x.IVOL)), axis = 1)
            
            candle[f"put_{strike_range}_today"] = candle.apply(lambda x : (BS.bs_put(x.close, 
                                                                                x[f"{strike_range}_strike_call_and_put"], 
                                                                                (x.current_week - 0.75 - self.end_holding_period/1440)/ 365, 
                                                                                Config.interest_rate, 
                                                                                x.IVOL)), axis = 1)
                
        self.price_matrix = candle[(candle.columns[candle.columns.str.contains('call')])| (candle.columns[candle.columns.str.contains('put')]) ]
        self.price_matrix["open"] = candle[['open']]
        self.price_matrix["current_week_yesterday"] = candle[['current_week_yesterday']]
        
    def _generate_return_matrix(self):
        
        self.return_matrix = pd.DataFrame({"date":sorted(self.price_matrix.index)}).set_index('date')
        for index, strike_range_lower in enumerate(self.strike_range[:-1]):
            for strike_range_upper in self.strike_range[index+1:]:

                call_long_yesterday = f"call_{strike_range_lower}_yesterday"
                call_long_today = f"call_{strike_range_lower}_today"
                call_short_yesterday = f"call_{strike_range_upper}_yesterday"
                call_short_today = f"call_{strike_range_upper}_today"
                call_short_strike = f"{strike_range_upper}_strike_call_and_put"
                call_long_strike = f"{strike_range_lower}_strike_call_and_put"
                
                put_long_yesterday = f"put_{strike_range_lower}_yesterday"
                put_long_today = f"put_{strike_range_lower}_today"
                put_short_yesterday = f"put_{strike_range_upper}_yesterday"
                put_short_today = f"put_{strike_range_upper}_today"
                put_short_strike = f"{strike_range_upper}_strike_call_and_put"
                put_long_strike = f"{strike_range_lower}_strike_call_and_put"
                
                if strike_range_lower<=0.75:
                    self.return_matrix[f"call_{strike_range_lower}_{strike_range_upper}"] = self.price_matrix.apply(lambda x: _calc_spread_return(x['open'],
                                                                                        x[call_long_yesterday], 
                                                                                        x[call_long_today], 
                                                                                        x[call_short_yesterday],
                                                                                        x[call_short_today], 
                                                                                        x[call_short_strike] - x[call_long_strike]),
                                                                                        axis=1)
                if strike_range_lower>=-0.75:
                    self.return_matrix[f"put_{strike_range_lower}_{strike_range_upper}"] = self.price_matrix.apply(lambda x: _calc_spread_return(x['open'], 
                                                                                         x[put_long_yesterday], 
                                                                                         x[put_long_today],
                                                                                         x[put_short_yesterday],
                                                                                         x[put_short_today],  
                                                                                         x[put_short_strike] - x[put_long_strike]),
                                                                                         axis=1)
        
    def _generate_performance_stats(self, 
                                    tail = None, 
                                    return_matrix_user = None, 
                                    start_holding_period = None, 
                                    end_holding_period = None):
        
        if start_holding_period is not None:
            self.start_holding_period = start_holding_period
        
        if end_holding_period is not None:
            self.end_holding_period = end_holding_period
            
        if return_matrix_user is not None:
            self.return_matrix = return_matrix_user[return_matrix_user.columns[(return_matrix_user.columns.str.contains('call')) | (return_matrix_user.columns.str.contains('put'))]]

        sim = pd.DataFrame(index = self.return_matrix.columns)
        
        if tail is not None:
            self.return_matrix = self.return_matrix.tail(tail)        
        return_matrix = self.return_matrix.copy()
        
        sim["Sharpe"] = 16 * return_matrix.mean()/return_matrix.std()

        rolling_cumulative = (1 + return_matrix).rolling(60).apply(lambda x : x.prod()).dropna()
        sim["60_Day_Median_Cumulative_Return"] = rolling_cumulative.median()
        sim["60_Day_Mean_Cumulative_Return"] = rolling_cumulative.mean()
        sim["60_Day_Max_Cumulative_Return"] = rolling_cumulative.max()
        sim["60_Day_Min_Cumulative_Return"] = rolling_cumulative.min()

        rolling_cumulative = (1 + return_matrix).rolling(250).apply(lambda x : x.prod()).dropna()
        sim["250_Day_Median_Cumulative_Return"] = rolling_cumulative.median()
        sim["250_Day_Mean_Cumulative_Return"] = rolling_cumulative.mean()
        sim["250_Day_Max_Cumulative_Return"] = rolling_cumulative.max()
        sim["250_Day_Min_Cumulative_Return"] = rolling_cumulative.min()

        sim["Return"] = ((1 + return_matrix).cumprod().tail(1) ** (1/len(return_matrix)) - 1).mean()

        sim["Last_60_Day_Cumulative_Return"] = (1 + return_matrix).tail(60).cumprod().tail(1).mean()
        sim["Last_250_Day_Cumulative_Return"] = (1 + return_matrix).tail(250).cumprod().tail(1).mean()
        sim["MDD"] = (((1 + return_matrix).cumprod() / ((1 + return_matrix).cumprod().cummax())) - 1).min()   
        
        sim["start_holding_period"] = self.start_holding_period
        sim["end_holding_period"] = self.end_holding_period
        
        self.performance_stats = sim.copy()

In [ ]:
start_date = '2015-01-09' #starting data point best left untouchedd
end_date = '2024-02-06'#'2022-01-03'#'2023-07-03' # Last day for which you want prediction - should be next trading day
prediction_date = '2024-01-23' # should be legitimate and after start date and before end date. Only dates >= are processed and pushed into DB
underlying = 'NIFTY' #NIFTY/BANKNIFTY
start_holding_period = 13
end_holding_period = 12
self = Gap_Move_Classifier(start_date, end_date, prediction_date, underlying, start_holding_period, end_holding_period)
self._generate_performance_stats()

In [ ]:
self.return_matrix["call_-1_0.75"]

In [2]:
start_holding_period = list(range(1,16))
end_holding_period = list(range(1,16))
tail = None
return_summary = []
summary = []
for start in start_holding_period:
    for end in end_holding_period:
        
        print(start,end)
        
        start_date = '2015-01-09' #starting data point best left untouchedd
        end_date = '2024-02-15'#'2022-01-03'#'2023-07-03' # Last day for which you want prediction - should be next trading day
        prediction_date = '2024-01-19' # should be legitimate and after start date and before end date. Only dates >= are processed and pushed into DB
        underlying = 'NIFTY' #NIFTY/BANKNIFTY
        self = Gap_Move_Classifier(start_date, end_date, prediction_date, underlying, start, end)
        self._generate_performance_stats(tail=tail)
        summary.append(self.performance_stats)
        
        temp = pd.merge(self.return_matrix, self.days_to_expiry[['current_week']], left_index = True, right_index = True)
        temp["start_holding_period"] = start
        temp["end_holding_period"] = end
        return_summary.append(temp)
        
summary = pd.concat(summary)
return_summary = pd.concat(return_summary)

1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
9 13
9 14
9 15
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
10 14
10 15
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
11 12
11 13
11 14
11 15
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
12 12
12 13
12 14
12 15
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
13 13
13 14
13 15
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
14 11
14 12
14 13
14 14
14 15
1

In [3]:
return_summary.current_week = np.where(return_summary.current_week == 5, 4, return_summary.current_week)
return_summary.current_week = np.where(return_summary.current_week == 6, 7, return_summary.current_week)
selected_strategies = []
for current_week in sorted(return_summary.current_week.unique()):
    
    print(current_week)
    temp = []
    for start in sorted(return_summary.start_holding_period.unique()):
        for end in sorted(return_summary.end_holding_period.unique()):
            
            self._generate_performance_stats(return_matrix_user = return_summary[(return_summary.current_week == current_week) & (return_summary.start_holding_period == start) & (return_summary.end_holding_period == end)], 
                                   start_holding_period = start, end_holding_period = end)
            temp.append(self.performance_stats)
    
    temp = pd.concat(temp)
    temp["current_week"] = current_week
    selected_strategies.append(temp.sort_values('Return', ascending=False).head(1))
selected_strategies = pd.concat(selected_strategies)
selected_strategies

1
2
3
4
7


,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period,current_week
call_-1_0.25,5.47200,6.45406,6.94338,18.79520,1.06037,1193.16519,1434.83380,5794.19145,191.15366,0.02912,1.95793,227.13514,-0.54130,1,4,1
call_-1_0.5,4.25194,3.61562,3.90893,15.20546,0.53269,398.73957,418.29596,1303.39338,43.24457,0.01834,1.17431,183.64280,-0.53777,13,15,2
call_-1_1,5.25993,4.03093,8.58952,57.72631,0.41679,720.29642,790.79854,4666.24064,22.20349,0.02386,4.08344,4666.24064,-0.71317,13,13,3
call_-1_-0.25,2.95075,1.73571,1.70213,3.04072,0.63215,7.83235,9.76267,21.22803,4.65713,0.00733,2.24006,5.66663,-0.43968,1,1,4
call_-1_1,1.94541,1.23056,1.35872,2.95609,0.56736,3.10943,3.21423,6.38145,1.75512,0.00600,1.60782,3.25701,-0.55949,13,13,7


In [25]:
selected_return_matrix = []
for strategy, start, end, current_week in zip(selected_strategies.index, 
                                              selected_strategies.start_holding_period, 
                                              selected_strategies.end_holding_period, 
                                              selected_strategies.current_week):
    selected_return_matrix.append(return_summary[(return_summary.start_holding_period == start) & 
                   (return_summary.end_holding_period == end) & 
                   (return_summary.current_week == current_week)][[strategy]].rename(columns = {strategy : "strategy_call_and_put"}))

selected_return_matrix = pd.concat(selected_return_matrix).sort_index()
self._generate_performance_stats(return_matrix_user=selected_return_matrix)
self.performance_stats

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
strategy_call_and_put,4.10084,2.93310,3.45095,12.31792,0.32050,65.82847,241.57662,2494.12223,3.31875,0.01716,5.07513,51.16222,-0.79960,15,15


In [5]:
for lag in [250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2228]:
    print(((1+selected_return_matrix.head(lag).tail(250)).cumprod()**(1/250)).tail(1))

            strategy_call_and_put
date                             
2016-01-18                1.01246
            strategy_call_and_put
date                             
2017-01-23                1.01355
            strategy_call_and_put
date                             
2018-01-25                1.02665
            strategy_call_and_put
date                             
2019-02-01                1.01187
            strategy_call_and_put
date                             
2020-02-10                1.01849
            strategy_call_and_put
date                             
2021-02-11                1.02249
            strategy_call_and_put
date                             
2022-02-17                1.02484
            strategy_call_and_put
date                             
2023-02-21                1.00802
            strategy_call_and_put
date                             
2024-01-25                1.01454


In [27]:
selected_return_matrix.describe()

,strategy_call_and_put
count,2242.00000
mean,0.02042
std,0.07969
min,-0.32583
25%,-0.02222
50%,0.02805
75%,0.07194
max,0.25515


In [29]:
selected_return_matrix = []
for strategy, start, end, current_week in zip(['call_-1_0.25', 'call_-1_0.25', 'call_-1_1', 'call_-1_-0.25',
       'call_-1_1'], [1, 9, 9, 9, 9], [4, 13, 1, 1, 10], [1, 2, 3, 4, 7]):
    selected_return_matrix.append(return_summary[(return_summary.start_holding_period == start) & 
                   (return_summary.end_holding_period == end) & 
                   (return_summary.current_week == current_week)][[strategy]].rename(columns = {strategy : "strategy_call_and_put"}))

selected_return_matrix = pd.concat(selected_return_matrix).sort_index()
self._generate_performance_stats(return_matrix_user=selected_return_matrix)
self.performance_stats

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
strategy_call_and_put,3.94834,2.69460,3.07830,10.20461,0.33297,45.46727,124.38001,1148.12323,3.23680,0.01585,5.56381,76.06534,-0.81050,15,15


In [30]:
for lag in [250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250]:
    print(((1+selected_return_matrix.head(lag).tail(250)).cumprod()**(1/250)).tail(1))

            strategy_call_and_put
date                             
2016-01-18                1.01135
            strategy_call_and_put
date                             
2017-01-23                1.01241
            strategy_call_and_put
date                             
2018-01-25                1.02551
            strategy_call_and_put
date                             
2019-02-01                1.01053
            strategy_call_and_put
date                             
2020-02-10                1.01435
            strategy_call_and_put
date                             
2021-02-11                1.01989
            strategy_call_and_put
date                             
2022-02-17                1.02052
            strategy_call_and_put
date                             
2023-02-21                1.01048
            strategy_call_and_put
date                             
2024-02-15                1.01748


In [9]:
temp = summary.copy()
for metric in ['Sharpe', 'MDD', '250_Day_Median_Cumulative_Return', '60_Day_Median_Cumulative_Return',
               'Return', 'Last_250_Day_Cumulative_Return', 'Last_60_Day_Cumulative_Return']:
    temp = temp[temp[metric]>=temp[metric].quantile(0.7)]
    
temp

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
put_-0.75_-0.25,4.30985,1.73701,1.74744,3.03384,0.60486,8.66974,10.44587,29.88222,2.15019,0.00872,2.28500,10.89297,-0.55074,9,3
call_-0.75_-0.25,4.07131,1.67174,1.68238,3.00185,0.57847,7.65389,8.82944,24.66960,2.00237,0.00811,2.22243,9.48060,-0.54583,9,4
call_-0.75_-0.25,4.11648,1.67117,1.69575,3.07914,0.58363,7.69677,9.08769,25.63750,2.14916,0.00820,2.21291,9.50961,-0.54588,9,5
put_-0.75_-0.25,4.44907,1.76480,1.79327,3.26861,0.59914,9.92193,11.64997,33.70265,1.93317,0.00908,2.20503,9.46315,-0.55071,13,4


In [10]:
summary.sort_values("Sharpe", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
put_-0.75_-0.5,4.54612,1.36597,1.35275,1.98168,0.66917,3.35428,3.55921,6.57636,1.55053,0.00486,1.53060,3.33301,-0.38226,1,10
put_-0.75_-0.25,4.53927,1.79616,1.82502,3.70383,0.51522,9.78991,12.70564,39.92221,2.29918,0.00928,2.23594,9.52320,-0.57959,1,10
put_-0.75_-0.25,4.52502,1.78463,1.81909,3.65140,0.52503,9.60095,12.56791,39.95484,2.49301,0.00924,2.23616,8.67212,-0.57172,1,9
put_-0.75_-0.5,4.52452,1.36212,1.35039,1.96960,0.67532,3.31491,3.53126,6.53632,1.60456,0.00484,1.52537,3.18289,-0.37672,1,9
put_-0.75_-0.5,4.51737,1.35247,1.34464,1.90373,0.70745,3.30492,3.45928,6.36937,1.62114,0.00479,1.52783,3.45610,-0.36635,1,5


In [11]:
summary.sort_values("MDD", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_-0.75,2.90858,1.18563,1.17408,1.58035,0.69421,1.86551,1.96145,3.53459,0.94388,0.00258,1.29651,1.90811,-0.34532,14,3
call_-1_-0.75,2.98974,1.19027,1.17906,1.59463,0.69582,1.90409,1.99537,3.70251,0.95386,0.00265,1.29425,1.96641,-0.34589,13,3
call_-1_-0.75,2.77997,1.17611,1.16623,1.57853,0.69266,1.80962,1.90653,3.56067,0.96438,0.00247,1.29756,1.91875,-0.34612,15,3
call_-1_-0.75,2.94157,1.19207,1.17753,1.59633,0.68840,1.89502,1.97984,3.53707,0.96225,0.00263,1.31130,1.95946,-0.34721,14,4
call_-1_-0.75,2.99479,1.19185,1.18172,1.61407,0.68936,1.92197,2.00424,3.61979,0.99613,0.00268,1.31984,1.99531,-0.34778,14,5


In [12]:
summary.sort_values("60_Day_Median_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.25,3.91060,2.61516,3.02020,12.79115,0.22747,38.87227,112.34332,910.85941,2.32570,0.01513,4.55129,36.87248,-0.84657,1,10
call_-1_0.25,3.89479,2.59725,2.99823,12.34514,0.23745,40.59687,111.43011,956.50599,2.69785,0.01505,4.47541,29.03176,-0.83662,1,9
call_-1_0.25,3.73684,2.59224,2.93915,11.21721,0.22441,28.83266,111.01009,795.37489,1.59508,0.01446,4.12191,24.78322,-0.83443,1,15
call_-1_0.25,3.73422,2.58880,2.97275,12.49763,0.25204,34.64663,107.26405,973.06055,1.47761,0.01455,4.20387,23.25511,-0.83029,13,15
call_-1_0.25,3.82470,2.57342,2.98232,12.18761,0.21970,32.49857,111.29211,867.55414,1.64368,0.01480,4.28024,28.20950,-0.84549,1,13


In [13]:
summary.sort_values("60_Day_Mean_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.5,3.44204,2.51031,3.14451,14.82802,0.19090,39.68116,135.23028,2021.33598,1.45043,0.01473,5.24075,30.22939,-0.88271,13,10
call_-1_0.5,3.32824,2.39276,3.13804,19.42577,0.19725,26.54495,222.37018,4287.66610,1.40572,0.01408,4.94320,23.84656,-0.88210,8,10
call_-1_0.5,3.44677,2.52559,3.12164,16.68615,0.16864,33.10184,141.53575,1496.04861,1.38062,0.01464,5.04307,32.01616,-0.88840,1,10
call_-1_0.5,3.42530,2.48404,3.11372,14.40822,0.20198,40.98626,131.81555,1781.83658,1.57879,0.01463,5.14435,23.00795,-0.87432,13,9
call_-1_0.5,3.34006,2.49765,3.10998,16.97640,0.18751,32.07638,132.20054,1916.52936,1.31595,0.01425,5.11535,21.20479,-0.87304,13,14


In [14]:
summary.sort_values("60_Day_Max_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_1,2.30047,1.70493,2.69920,26.37272,0.12918,5.56647,220.20563,6897.54175,0.27421,0.00940,4.32546,6.10177,-0.92282,8,6
call_-1_1,2.42541,1.96834,2.94441,25.95548,0.11586,7.34102,270.62656,8417.45987,0.26651,0.01024,4.78500,8.28268,-0.92337,8,14
call_-1_1,2.50430,1.97912,2.95506,25.79045,0.11842,9.34341,292.73939,9016.98883,0.36223,0.01070,4.84255,11.47455,-0.93030,8,10
call_-1_1,2.32883,1.72836,2.77148,24.95951,0.13598,7.27575,233.80788,8063.03395,0.33640,0.00958,4.24024,5.30010,-0.91948,8,7
call_-1_1,2.37782,1.79870,2.80818,24.89755,0.12019,8.55662,221.23534,6859.81638,0.30796,0.00989,4.24004,6.01375,-0.92300,8,8


In [15]:
summary.sort_values("60_Day_Min_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
put_-0.75_-0.5,4.30807,1.33851,1.32752,1.86220,0.73713,3.07341,3.32637,6.71745,1.32728,0.00457,1.48223,3.19140,-0.35799,8,3
put_-0.75_-0.5,4.38124,1.34948,1.33593,1.82786,0.73360,3.16031,3.38101,6.07184,1.34627,0.00468,1.48819,3.25496,-0.35654,13,3
put_-0.75_-0.5,4.43740,1.34433,1.34077,1.96159,0.73303,3.22565,3.44475,6.98124,1.45764,0.00473,1.50553,3.35312,-0.36076,8,5
put_-0.75_-0.5,4.28682,1.34199,1.32856,1.80431,0.73183,3.09696,3.30762,6.06022,1.32462,0.00459,1.48885,3.12172,-0.35566,14,3
put_-0.75_-0.5,4.38902,1.34096,1.33541,1.88536,0.73086,3.18305,3.39326,6.73158,1.40261,0.00467,1.50366,3.32787,-0.35998,8,4


In [16]:
summary.sort_values("250_Day_Median_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.25,3.87683,2.53589,3.02234,11.28706,0.26362,48.23275,108.34296,950.30227,2.42879,0.01511,4.52859,27.00971,-0.83082,13,9
call_-1_0.25,3.89360,2.56942,3.04805,11.63630,0.25186,46.48182,109.83732,1062.42930,2.20842,0.01520,4.56115,33.90179,-0.84022,13,10
call_-1_0.25,3.76377,2.43245,2.95177,10.59977,0.24765,43.64023,99.46206,822.41902,1.85732,0.01462,4.34645,22.86473,-0.83079,13,8
call_-1_0.25,3.73824,2.50585,2.94355,11.26837,0.26383,42.94440,93.86620,927.36404,1.53811,0.01454,4.05899,23.51662,-0.83477,13,11
call_-1_0.25,3.77276,2.53271,2.93273,10.74519,0.25963,42.84998,96.76011,754.91563,2.19533,0.01466,4.48182,21.31966,-0.83041,14,9


In [17]:
summary.sort_values("250_Day_Mean_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_1,2.50430,1.97912,2.95506,25.79045,0.11842,9.34341,292.73939,9016.98883,0.36223,0.01070,4.84255,11.47455,-0.93030,8,10
call_-1_0.75,2.88948,2.21337,3.08957,23.35316,0.15180,16.02295,275.91493,7032.59343,0.78315,0.01258,5.04960,17.13923,-0.91058,8,10
call_-1_1,2.42541,1.96834,2.94441,25.95548,0.11586,7.34102,270.62656,8417.45987,0.26651,0.01024,4.78500,8.28268,-0.92337,8,14
call_-1_1,2.48608,1.93176,2.90344,24.25108,0.12746,9.84735,267.20963,7755.10294,0.41287,0.01058,4.67777,8.21678,-0.92393,8,9
call_-1_1,2.38134,1.99391,2.88620,24.16556,0.11874,6.35002,262.35747,8157.94541,0.22199,0.00997,4.38937,7.71530,-0.93055,8,15


In [18]:
summary.sort_values("250_Day_Max_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_1,2.50430,1.97912,2.95506,25.79045,0.11842,9.34341,292.73939,9016.98883,0.36223,0.01070,4.84255,11.47455,-0.93030,8,10
call_-1_1,2.42541,1.96834,2.94441,25.95548,0.11586,7.34102,270.62656,8417.45987,0.26651,0.01024,4.78500,8.28268,-0.92337,8,14
call_-1_1,2.38134,1.99391,2.88620,24.16556,0.11874,6.35002,262.35747,8157.94541,0.22199,0.00997,4.38937,7.71530,-0.93055,8,15
call_-1_1,2.45577,1.96373,2.92303,23.64276,0.11417,7.85388,259.15039,8076.70866,0.29470,0.01043,4.62130,8.98564,-0.92947,8,13
call_-1_1,2.32883,1.72836,2.77148,24.95951,0.13598,7.27575,233.80788,8063.03395,0.33640,0.00958,4.24024,5.30010,-0.91948,8,7


In [19]:
summary.sort_values("250_Day_Min_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0,4.21722,2.42592,2.59878,7.79722,0.34335,31.05398,55.90378,338.85567,4.07673,0.01387,3.99800,33.26352,-0.76965,9,9
call_-1_0,4.23095,2.45210,2.61797,7.97942,0.33183,31.88956,57.07143,371.29511,3.71122,0.01392,4.00022,39.14239,-0.77721,9,10
call_-1_0,4.02218,2.32240,2.44529,6.29607,0.35469,25.72324,41.88156,205.42448,3.51810,0.01305,3.21189,24.69957,-0.76703,13,1
call_-1_0,4.03729,2.25783,2.50463,8.09041,0.34159,26.80588,49.77733,276.07987,3.45586,0.01318,3.05563,14.10301,-0.76695,3,9
call_-1_0,4.26185,2.32596,2.58765,7.96150,0.36468,29.28292,55.37204,330.45139,3.40055,0.01382,3.59714,31.49317,-0.76290,1,5


In [20]:
summary.sort_values("Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.25,3.89360,2.56942,3.04805,11.63630,0.25186,46.48182,109.83732,1062.42930,2.20842,0.01520,4.56115,33.90179,-0.84022,13,10
call_-1_0.25,3.91060,2.61516,3.02020,12.79115,0.22747,38.87227,112.34332,910.85941,2.32570,0.01513,4.55129,36.87248,-0.84657,1,10
call_-1_0.25,3.87683,2.53589,3.02234,11.28706,0.26362,48.23275,108.34296,950.30227,2.42879,0.01511,4.52859,27.00971,-0.83082,13,9
call_-1_0.25,3.89479,2.59725,2.99823,12.34514,0.23745,40.59687,111.43011,956.50599,2.69785,0.01505,4.47541,29.03176,-0.83662,1,9
call_-1_0.25,3.81635,2.56731,3.00728,12.28583,0.24389,39.89427,106.26470,1013.24240,2.16311,0.01489,4.35322,26.38614,-0.83909,13,13


In [21]:
summary.sort_values("Last_60_Day_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.75,2.92377,2.28887,2.90329,15.97416,0.14577,21.35931,124.27086,2672.94314,0.79383,0.01278,6.32773,38.14957,-0.91143,9,10
call_-1_1,2.53509,2.05047,2.72807,16.86901,0.11366,12.56765,114.39508,3098.75829,0.36672,0.01089,6.21187,27.11729,-0.93100,9,10
call_-1_0.75,2.83248,2.26731,2.87617,17.77555,0.14164,16.59681,118.13582,2500.95032,0.61622,0.01230,6.15870,26.85233,-0.90304,9,14
call_-1_0.75,2.90800,2.26740,2.86361,15.14361,0.15576,20.10798,117.31976,2331.95810,0.88829,0.01267,6.14533,28.01050,-0.90395,9,9
call_-1_1,2.45484,2.02565,2.71165,19.14459,0.11033,9.90282,107.20374,2880.26367,0.27315,0.01043,6.12526,19.52229,-0.92379,9,14


In [22]:
summary.sort_values("Last_250_Day_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.25,3.82936,2.56701,2.93406,11.16115,0.25032,39.28712,96.95182,957.13838,3.02757,0.01483,5.16737,51.12520,-0.84080,9,10
call_-1_0.25,3.74459,2.42972,2.74887,9.41357,0.30728,34.75911,72.65034,570.90829,2.70228,0.01428,4.86345,50.96439,-0.82302,9,4
call_-1_0.25,3.77364,2.41896,2.80069,9.84569,0.30751,35.09875,78.97183,678.08500,3.04060,0.01441,4.87864,50.44394,-0.82381,9,5
call_-1_0.5,3.36640,2.48029,3.00428,14.01856,0.18964,33.00619,120.54436,1838.20432,1.66873,0.01427,6.00846,48.74940,-0.88315,9,10
call_-1_0.5,3.25883,2.34577,2.75110,11.35408,0.23955,26.85071,80.98036,996.80156,1.71254,0.01353,5.37492,44.61971,-0.86689,9,4


In [ ]:
# summary.to_csv('Nifty_Full.csv')

In [ ]:
return_summary.to_csv('Nifty_Full_Sim.csv')

In [ ]:
(1 + self.return_matrix["call_-1_0.75"]).rolling(250).apply(lambda x: x.prod()).dropna().quantile(0.0025)

In [ ]:
27357.49229**(1/1000)

In [ ]:
import numpy as np
future = 21393
future*=np.exp(0.07*14/248)

In [ ]:
future*0.99

In [ ]:
future*1.005